<a href="https://colab.research.google.com/github/cateberry/dissertation/blob/dissertation/Copy_of_MPC_CNN_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT) 

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'My Drive/dissertation' 
# replace with your Github username 
GIT_USERNAME = "cateberry" 
# definitely replace with your
GIT_TOKEN = "7dade44469ddae48962f5d81a3b257589ed1b2c9"
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "dissertation"

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
# !mkdir "{PROJECT_PATH}"

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/My Drive/dissertation
GIT_PATH:  https://7dade44469ddae48962f5d81a3b257589ed1b2c9@github.com/cateberry/dissertation.git


In [16]:
%cd "{PROJECT_PATH}"
!git clone "{GIT_PATH}"

/content/drive/My Drive/dissertation
fatal: destination path 'dissertation' already exists and is not an empty directory.


In [ ]:
%cd /content/drive/My\ Drive/dissertation
!python setup.py build_ext --inplace

/content/drive/My Drive/dissertation/dissertation
Compiling image_analysis/im2col/im2col_cython_float.pyx because it changed.
Compiling image_analysis/im2col/im2col_cython_object.pyx because it changed.
[1/2] Cythonizing image_analysis/im2col/im2col_cython_float.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/My Drive/dissertation/dissertation/image_analysis/im2col/im2col_cython_float.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing image_analysis/im2col/im2col_cython_object.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/My Drive/dissertation/dissertation/image_analysis/im2col/im2col_cython_object.pyx
  tree = Parsing.p_module(s, pxd, full_

In [4]:
%cd /content/drive/My\ Drive/dissertation/dissertation/image_analysis
%ls

/content/drive/My Drive/dissertation/dissertation/image_analysis
'Base MPC implementation.py'   'MPC_CNN_COLAB (1).ipynb'
 Convnet.py                     MPC_CNN_COLAB.ipynb
'Copy of MPC_CNN_COLAB.ipynb'  'MPC CNN.py'
 im2col/                        pond/
 __init__.py                   'Private Image Analysis.ipynb'


In [6]:
import keras
import numpy as np
import time
import pond
from pond.tensor import NativeTensor, PublicEncodedTensor, PrivateEncodedTensor
from pond.nn import Dense, Relu, Reveal, CrossEntropy, SoftmaxStable, Sequential, DataLoader, Conv2D, \
    AveragePooling2D, Flatten, BatchNorm, ReluNormal, ReluGalois, Conv2DQuant, DenseQuant
from keras.utils import to_categorical


np.random.seed(42)

# set errors error behaviour for overflow/underflow
_ = np.seterr(over='raise')
_ = np.seterr(under='raise')
_ = np.seterr(invalid='raise')

# %%
"""
- Load data from Keras
- Split into training and test data
- Normalise to [0,1] (greyscale images with values in [0,255])
- Transform targets to one-hot representation
"""

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype(np.uint8)
x_test = x_test.astype(np.uint8)
x_train = x_train[:, np.newaxis, :, :] / 255.0
x_test = x_test[:, np.newaxis, :, :] / 255.0
# x_train = x_train[:, np.newaxis, :, :]
# x_test = x_test[:, np.newaxis, :, :]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

"""
Split into train-test-val
Train in usual way using train and val
Write separate function for testing on test set (completely unseen)
Purpose of testing function is for evaluating quantized network
Compare testing of quantized network with non-quantized network 
Need a way to save the parameters of the trained MPC network 
"""

# Size of pooling area for max pooling
# pool_size = (2, 2)
# Convolution kernel size (kernel_height, kernel_width, input_channels, num_filters)
# kernel_size = (5, 5, 1, 16)

# convnet_shallow_gal = Sequential([
#     #Conv2D((3, 3, 1, 32), strides=1, padding=1, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp)),
#     Conv2DQuant((3, 3, 1, 32), strides=1, padding=1, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp)),
#     BatchNorm(),
#     ReluGalois(order=4, mu=0.0, sigma=1.0),     # TODO: investigate overflow error
#     AveragePooling2D(pool_size=(2, 2)),
#     Flatten(),
#     # Dense(10, 6272),  # 3136 5408 6272
#     DenseQuant(10, 6272),
#     Reveal(),
#     SoftmaxStable()
# ])

convnet_shallow = Sequential([
    Conv2D((3, 3, 1, 32), strides=1, padding=1, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp)),
    BatchNorm(),
    ReluNormal(order=4, mu=0.0, sigma=1.0, approx_type='lagrange-uniform'), # approx_type='taylor'),
    # Relu(order=4),
    AveragePooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(10, 6272),  # 3136 5408 6272
    Reveal(),
    SoftmaxStable()
])


def accuracy(classifier, x, y, verbose=0, wrapper=NativeTensor):
    predicted_classes = classifier \
        .predict(DataLoader(x, wrapper), verbose=verbose).reveal() \
        .argmax(axis=1)

    correct_classes = NativeTensor(y) \
        .argmax(axis=1)

    matches = predicted_classes.unwrap() == correct_classes.unwrap()
    return sum(matches) / len(matches)


# %%
"""
Train on different types of Tensor
"""
# NativeTensor (like plaintext)
x_train = x_train[:512]
y_train = y_train[:512]
x_test = x_test[:128]
y_test = y_test[:128]

tensortype = PrivateEncodedTensor  # TODO: Change back to NativeTensor
batch_size = 32
input_shape = [batch_size] + list(x_train.shape[1:])
epochs = 3
learning_rate = 0.01

convnet_shallow.initialize(initializer=tensortype, input_shape=input_shape)

start = time.time()
convnet_shallow.fit(
    x_train=DataLoader(x_train, wrapper=tensortype),
    y_train=DataLoader(y_train, wrapper=tensortype),
    x_valid=DataLoader(x_test, wrapper=tensortype),
    y_valid=DataLoader(y_test, wrapper=tensortype),
    loss=CrossEntropy(),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    learning_rate=learning_rate
)
end = time.time()
time_taken = end - start

print("Elapsed time: ", time_taken)

raise Exception()


[(-0.5*x - 0.5)*(-0.25*x + 0.25)*(-0.166666666666667*x + 0.5), (-0.5*x + 0.5)*(-0.25*x + 0.75)*(0.5*x + 1.5), (-0.5*x + 1.5)*(0.25*x + 0.75)*(0.5*x + 0.5), (0.166666666666667*x + 0.5)*(0.25*x + 0.25)*(0.5*x - 0.5)] [-3.0, -1.0, 1.0, 3.0]
[0.375000000000000 0.125000000000000 0.500000000000000
 -1.17961196366423e-16]
2020-08-19 10:04:08.760188 Epoch 1/3
512/512 [============================>..] - ETA: 0:04:13 - train_loss: 1.12109 - train_acc 0.68750 - val_loss 1.07520 - val_acc 0.73438
2020-08-19 11:07:30.582443 Epoch 2/3
512/512 [============================>..] - ETA: 0:04:16 - train_loss: 0.81348 - train_acc 0.75000 - val_loss 0.85278 - val_acc 0.78906
2020-08-19 12:11:36.067493 Epoch 3/3
512/512 [============================>..] - ETA: 0:04:18 - train_loss: 0.68726 - train_acc 0.75000 - val_loss 0.76392 - val_acc 0.78906

Elapsed time:  11521.919048786163


Exception: ignored